In [24]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection


import datetime, nltk, warnings 
from wordcloud import WordCloud, STOPWORDS 

In [27]:
import os
os.environ['OMP_NUM_THREADS'] = '1'


In [ ]:
from sklearn.cluster import DBSCAN

In [8]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
iris = load_iris()  
km = KMeans(n_clusters=2) 
y = km.fit_predict(iris.data) #clustering on data 
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


C:\Users\CCC\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [ ]:
df_initial = pd.read_csv('C:/Users/CCC/Documents/Data analysis/data.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})
print('Dataframe shape:', df_initial.shape)
df_initial.head(2)

In [29]:
df_initial['InvoiceDate'] = pd.to_datetime(df_initial['InvoiceDate'])
df_initial.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [11]:
print("no. missing values\n", df_initial.isna().sum())

no. missing values
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [12]:
df_initial.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', df_initial.shape)
print("no. missing values\n", df_initial.isna().sum())

Dataframe dimensions: (406829, 8)
no. missing values
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [13]:
print('duplicate entries: {}'.format(df_initial.duplicated().sum()))
df_initial.drop_duplicates(inplace = True)
print('Dataframe shape:', df_initial.shape)

duplicate entries: 5225
Dataframe shape: (401604, 8)


In [14]:
df_number = pd.DataFrame([{'products': len(df_initial['StockCode'].value_counts()), 
                           'transactions': len(df_initial['InvoiceNo'].value_counts()),
                           'customers': len(df_initial['CustomerID'].value_counts())}], 
                         columns = ['products', 'transactions', 'customers'], index = ['quantity'])
df_number.head()

,products,transactions,customers
quantity,3684,22190,4372


In [15]:
temp = df_initial.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
no_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
no_products_per_basket[:10].sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of products
0,12346,541431,1
1,12346,C541433,1
2,12347,537626,31
3,12347,542237,29
4,12347,549222,24
5,12347,556201,18
6,12347,562032,22
7,12347,573511,47
8,12347,581180,11
9,12348,539318,17


In [16]:
print(df_initial.sort_values('CustomerID')[:5])

       InvoiceNo StockCode                         Description  Quantity  \
61619     541431     23166      MEDIUM CERAMIC TOP STORAGE JAR     74215   
61624    C541433     23166      MEDIUM CERAMIC TOP STORAGE JAR    -74215   
286623    562032     22375   AIRLINE BAG VINTAGE JET SET BROWN         4   
72260     542237     84991         60 TEATIME FAIRY CAKE CASES        24   
14943     537626     22772  PINK DRAWER KNOB ACRYLIC EDWARDIAN        12   

               InvoiceDate  UnitPrice CustomerID         Country  
61619  2011-01-18 10:01:00       1.04      12346  United Kingdom  
61624  2011-01-18 10:17:00       1.04      12346  United Kingdom  
286623 2011-08-02 08:48:00       4.25      12347         Iceland  
72260  2011-01-26 14:30:00       0.55      12347         Iceland  
14943  2010-12-07 14:57:00       1.25      12347         Iceland  


In [17]:
df_check = df_initial[df_initial['Quantity'] < 0][['CustomerID','Quantity',
                                                   'StockCode','Description','UnitPrice']]
#print(df_check.head(5))
for index, col in  df_check.iterrows():
    if df_initial[(df_initial['CustomerID'] == col[0]) & (df_initial['Quantity'] == -col[1]) 
                & (df_initial['Description'] == col[2])].shape[0] == 0: 
        print(index, df_check.loc[index])
        print(15*'-'+'>'+' HYPOTHESIS NOT FULFILLED')
        break

141 CustomerID        14527
Quantity             -1
StockCode             D
Description    Discount
UnitPrice          27.5
Name: 141, dtype: object
---------------> HYPOTHESIS NOT FULFILLED


C:\Users\CCC\AppData\Local\Temp\ipykernel_4008\68061424.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df_initial[(df_initial['CustomerID'] == col[0]) & (df_initial['Quantity'] == -col[1])
C:\Users\CCC\AppData\Local\Temp\ipykernel_4008\68061424.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  & (df_initial['Description'] == col[2])].shape[0] == 0:


In [18]:
df_check = df_initial[(df_initial['Quantity'] < 0) & (df_initial['Description'] != 'Discount')][
                                 ['CustomerID','Quantity','StockCode',
                                  'Description','UnitPrice']]
print(df_check.head(5))
for index, col in  df_check.iterrows():
    if df_initial[(df_initial['CustomerID'] == col[0]) & (df_initial['Quantity'] == -col[1]) 
                & (df_initial['Description'] == col[2])].shape[0] == 0: 
        print(index, df_check.loc[index])
        print(15*'-'+'>'+' HYPOTHESIS NOT FULFILLED')
        break

    CustomerID  Quantity StockCode                        Description  \
154      15311        -1    35004C    SET OF 3 COLOURED  FLYING DUCKS   
235      17548       -12     22556     PLASTERS IN TIN CIRCUS PARADE    
236      17548       -24     21984   PACK OF 12 PINK PAISLEY TISSUES    
237      17548       -24     21983   PACK OF 12 BLUE PAISLEY TISSUES    
238      17548       -24     21980  PACK OF 12 RED RETROSPOT TISSUES    

     UnitPrice  
154       4.65  
235       1.65  
236       0.29  
237       0.29  
238       0.29  
154 CustomerID                               15311
Quantity                                    -1
StockCode                               35004C
Description    SET OF 3 COLOURED  FLYING DUCKS
UnitPrice                                 4.65
Name: 154, dtype: object
---------------> HYPOTHESIS NOT FULFILLED


C:\Users\CCC\AppData\Local\Temp\ipykernel_4008\290411021.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df_initial[(df_initial['CustomerID'] == col[0]) & (df_initial['Quantity'] == -col[1])
C:\Users\CCC\AppData\Local\Temp\ipykernel_4008\290411021.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  & (df_initial['Description'] == col[2])].shape[0] == 0:


In [ ]:
df_cleaned.shape

In [ ]:
df_initial.head()

In [ ]:
list_keywords = []
for k,v in count_keywords.items():
    list_keywords.append([keywords_select[k],v])
list_keywords.sort(key = lambda x:x[1], reverse = True)
print(len(list_keywords))

In [ ]:
fig, ax = plt.subplots(figsize=(17, 25))
y_axis = [i[1] for i in list_keywords[:125]]
x_axis = [k for k,i in enumerate(list_keywords[:125])]
x_label = [i[0] for i in list_keywords[:125]]
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 13)
plt.yticks(x_axis, x_label)
plt.xlabel("No. of occurences", fontsize = 18, labelpad = 10)
ax.barh(x_axis, y_axis, align = 'center')
ax = plt.gca()
ax.invert_yaxis()
#_______________________________________________________________________________________
plt.title("Words occurence",bbox={'facecolor':'k', 'pad':5}, color='w',fontsize = 25)
plt.show()

In [ ]:
n_clusters = 5
kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
kmeans.fit(X)
clusters = kmeans.predict(X)
print(pd.Series(clusters).value_counts())
from yellowbrick.cluster import SilhouetteVisualizer  
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure 
print(visualizer.silhouette_score_)  